# Assignment as a min cost flow 00

This example uses the more general _minimum cost flow_ solve.

This requires different node numbering than the original assignment example, because the minimum cost solver requires all nodes to have distinct numbering.

In [2]:
from ortools.graph import pywrapgraph
import time

## Define the directed graph for the flow. These are divided into three sub-arrays:

1. Arcs leading out of the source
2. Flows between workers and tasks, for which there are non-zero costs
3. Arce leading into the sink.

The data also includes the vector `supplies`: the supply at each node $0-9$.

The condition of _flow in equals flow out_ forces the flow out of each worker to be $1$. This also forces each task to be be assigned to a worker for this configuration.


In [11]:
def main():
    """
    Solve an assignment problem with MinCostFlow
    """
    
    # instantiate a simple SimpleMinCostFlow solver
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    
    # define the directed graph for the flow
    start_nodes =  [0, 0, 0, 0] + [ 1,  1,  1,  1,  2,  2,  2,  2,   3,  3,  3,   3,  4,   4,  4,   4] + [5, 6, 7, 8]
    end_nodes   =  [1, 2, 3, 4] + [ 5,  6,  7,  8,  5,  6,  7,  8,   5,  6,  7,   8,  5,   6,  7,   8] + [9, 9, 9, 9]
    capacities  =  [1, 1, 1, 1] + [ 1,  1,  1,  1,  1,  1,  1,  1,   1,  1,  1,   1,  1,   1,  1,   1] + [1, 1, 1, 1]
    costs       = ([0, 0, 0, 0] + [90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115] + [0, 0, 0, 0])

    supplies = [4, 0, 0, 0, 0, 0, 0, 0, 0, -4]
    source = 0
    sink = 9
    tasks = 4
    
    # add each arc
    for i in range(len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacities[i], costs[i])
    
    # add node supplies
    for i in range(len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])
    
    # invoke the solver and display the solution
    if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
        print('Total cost = %d.' % (min_cost_flow.OptimalCost()))
        print()
        for arc in range(min_cost_flow.NumArcs()):
            
            # can ignore the nodes from source and to sink
            if min_cost_flow.Tail(arc) != source and min_cost_flow.Head(arc) != sink:
                
                # arcs in the solution have a flow value of 1. Their start and
                # end nodes give an assignment of worker to task.
                if min_cost_flow.Flow(arc) > 0:
                    print('Worker %d assigned to task %d; cost = %d.' % (
                    min_cost_flow.Tail(arc),
                    min_cost_flow.Head(arc),
                    min_cost_flow.UnitCost(arc)))
                    
    else:
        print('There was an issue; no optimal solution found.')   

In [12]:
if __name__ == "__main__":
    start_time = time.clock()
    main()
    print()
    print("Time = ", time.clock() - start_time)

Total cost = 265.

Worker 1 assigned to task 8; cost = 70.
Worker 2 assigned to task 7; cost = 55.
Worker 3 assigned to task 6; cost = 95.
Worker 4 assigned to task 5; cost = 45.

Time =  0.0005429999999995161
